HW2 - KNN algorithm - Sara Saremi - 97243045

In [ ]:
import numpy as np
import pandas as pd
import os

os.getcwd()

'/content'

In [ ]:
os.listdir(os.getcwd())

['.config', 'adult.test', 'adult.data', 'sample_data']

loading data...

In [ ]:
pd.read_csv("adult.data")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [ ]:
col_list = ["age", "workclass",	 "fnlwgt",
            "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "hours-per-week", "native-country", "class"]
data = pd.read_csv("adult.data", sep=', ', engine='python', usecols=col_list)

***dealing with missing data***
 
Most subjects lack data for capital-loss and capital-gain therefore we eliminate these two columns from our data.
 
Other missing datas are from "workclass", "occupation" and "native-country".

In [ ]:
def most_frequent(List): 
  max = 0
  res = List[0] 
  for i in List: 
      freq = List.count(i) 
      if freq > max: 
          max = freq 
          res = i 
  return res

In [ ]:
l1 = data['native-country'].tolist()
frequent_country = most_frequent(l1)
print("most frequent country: ",frequent_country) 
l2 = data['occupation'].tolist()
frequent_occupation = most_frequent(l2)
print("most frequent occupation: ",frequent_occupation) 
l3 = data['workclass'].tolist()
frequent_workclass = most_frequent(l3)
print("most frequent workclass: ",frequent_workclass) 

most frequent country:  United-States
most frequent occupation:  Prof-specialty
most frequent workclass:  Private


***One-hot encoding***

For categorical values we use one-hot encoding as implemented below



In [ ]:
workclass_types = ["Private", "Self-emp-not-inc", "Self-emp-inc", "Federal-gov", "Local-gov", "State-gov", "Without-pay", "Never-worked"]
marital_types = ["Married-civ-spouse", "Divorced", "Never-married", "Separated", "Widowed", "Married-spouse-absent", "Married-AF-spouse"]
occupation_types = ["Tech-support", "Craft-repair", "Other-service", "Sales", "Exec-managerial", "Prof-specialty", "Handlers-cleaners", "Machine-op-inspct", "Adm-clerical", "Farming-fishing", "Transport-moving", "Priv-house-serv", "Protective-serv", "Armed-Forces"]
relationship_types = ["Wife", "Own-child", "Husband", "Not-in-family", "Other-relative", "Unmarried"]
sex_types = ["Female", "Male"]
country_types = ["United-States", "Cambodia", "England", "Puerto-Rico", "Canada", "Germany", "Outlying-US(Guam-USVI-etc)", "India", 'Japan', "Greece", "South", "China", "Cuba", "Iran", "Honduras", "Philippines", "Italy", "Poland", "Jamaica", "Vietnam", "Mexico", "Portugal", "Ireland", "France", "Dominican-Republic", "Laos", "Ecuador", "Taiwan", "Haiti", "Columbia", "Hungary", "Guatemala", "Nicaragua", "Scotland", "Thailand", "Yugoslavia", "El-Salvador", "Trinadad&Tobago", "Peru", "Hong", "Holand-Netherlands"]
race_types = ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black']
education_types = ["Bachelors", "Some-college", "11th", "HS-grad", "Prof-school", "Assoc-acdm", "Assoc-voc", "9th", "7th-8th", "12th", "Masters", "1st-4th", "10th", "Doctorate", "5th-6th", "Preschool"]

In [ ]:
def one_hot_encoder(types, total, category_key):
  mapping = {}
  for x in range(len(types)):
    mapping[types[x]] = x

  one_hot_encode = []

  for t in total:
    if t == '?':
      if category_key == 1: 
        t = frequent_country
      elif category_key == 2:
        t = frequent_occupation
      elif category_key == 3:
        t = frequent_workclass
    arr = list(np.zeros(len(types), dtype = int))
    arr[mapping[t]] = 1
    one_hot_encode.append(arr)

  return one_hot_encode

A sample subject before one-hot encoding with missing data:

In [ ]:
sample = data.iloc[61]
print(sample)

age                                  32
workclass                             ?
fnlwgt                           293936
education                       7th-8th
education-num                         4
marital-status    Married-spouse-absent
occupation                            ?
relationship              Not-in-family
race                              White
sex                                Male
hours-per-week                       40
native-country                        ?
class                             <=50K
Name: 61, dtype: object


In [ ]:
data['education'] = one_hot_encoder(education_types, data['education'], 0)
data['workclass'] = one_hot_encoder(workclass_types, data['workclass'], 3)
data['marital-status'] = one_hot_encoder(marital_types, data['marital-status'], 0)
data['occupation'] = one_hot_encoder(occupation_types, data['occupation'], 2)
data['relationship'] = one_hot_encoder(relationship_types, data['relationship'], 0)
data['sex'] = one_hot_encoder(sex_types, data['sex'], 0)
data['native-country'] = one_hot_encoder(country_types, data['native-country'], 1)
data['race'] = one_hot_encoder(race_types, data['race'], 0)

same sample after encoding and handling missing values:



In [ ]:
sample = data.iloc[61]
print(sample)

age                                                              32
workclass                                  [1, 0, 0, 0, 0, 0, 0, 0]
fnlwgt                                                       293936
education          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
education-num                                                     4
marital-status                                [0, 0, 0, 0, 0, 1, 0]
occupation               [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
relationship                                     [0, 0, 0, 1, 0, 0]
race                                                [1, 0, 0, 0, 0]
sex                                                          [0, 1]
hours-per-week                                                   40
native-country    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
class                                                         <=50K
Name: 61, dtype: object


***Normalization***

Now we should normalize numeric datas.

In [ ]:
def normalization(numeric_col):
  min_val = min(numeric_col)
  max_val = max(numeric_col)
  for i in range(len(numeric_col)):
    numeric_col[i] = (numeric_col[i]-min_val)/(max_val - min_val)

  return numeric_col


In [ ]:
data['age'] = normalization(data['age'].tolist())
data['fnlwgt'] = normalization(data['fnlwgt'].tolist())
data['education-num'] = normalization(data['education-num'].tolist())
data['hours-per-week'] = normalization(data['hours-per-week'].tolist())

Same sample data with normalized values.

In [ ]:
sample = data.iloc[61]
print(sample)

age                                                        0.205479
workclass                                  [1, 0, 0, 0, 0, 0, 0, 0]
fnlwgt                                                     0.191284
education          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
education-num                                                   0.2
marital-status                                [0, 0, 0, 0, 0, 1, 0]
occupation               [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
relationship                                     [0, 0, 0, 1, 0, 0]
race                                                [1, 0, 0, 0, 0]
sex                                                          [0, 1]
hours-per-week                                             0.397959
native-country    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
class                                                         <=50K
Name: 61, dtype: object


We repeat the same process for test datas.

In [ ]:
test_data = pd.read_csv("adult.test", sep=', ', engine='python', usecols=col_list)

test_data['education'] = one_hot_encoder(education_types, test_data['education'], 0)
test_data['workclass'] = one_hot_encoder(workclass_types, test_data['workclass'], 3)
test_data['marital-status'] = one_hot_encoder(marital_types, test_data['marital-status'], 0)
test_data['occupation'] = one_hot_encoder(occupation_types, test_data['occupation'], 2)
test_data['relationship'] = one_hot_encoder(relationship_types, test_data['relationship'], 0)
test_data['sex'] = one_hot_encoder(sex_types, test_data['sex'], 0)
test_data['native-country'] = one_hot_encoder(country_types, test_data['native-country'], 1)
test_data['race'] = one_hot_encoder(race_types, test_data['race'], 0)

test_data['age'] = normalization(test_data['age'].tolist())
test_data['fnlwgt'] = normalization(test_data['fnlwgt'].tolist())
test_data['education-num'] = normalization(test_data['education-num'].tolist())
test_data['hours-per-week'] = normalization(test_data['hours-per-week'].tolist())

***KNN Algorithm***

Now we can start the KNN algorithm implementation.

A function for calculating one-hot variable distances. Since one-hot variables are lists we couldn't calculate them same as other features.


In [ ]:
def one_hot_distance(a , b):
  if a == b:
    return 0
  return 2

In [ ]:
def euclidean_distance(i , j):
  total_dist = 0
  b = test_data.iloc[i]
  a = data.iloc[j]
  for k in range(len(a)-1):
    if type(a[k]) == list:
      total_dist += one_hot_distance(a[k] , b[k])
    else:
      total_dist += (a[k]-b[k])**2
  return total_dist



In [ ]:
def predict(K):
        final_output = []
        for i in range(len(test_data)):
            distances = []
            labels = []
            for j in range(len(data)):
                dist = euclidean_distance(i , j)
                distances.append([dist, j])
            distances.sort()
            distances = distances[0:K]
            for distances, j in distances:
                class_value = data.iloc[j]
                labels.append(class_value[12])
            ans = most_frequent(labels)
            final_output.append(ans)
            if i % 50 == 0:
              print("algorithm is working...")
              print(i, " " , ans)

            
        return final_output

In [ ]:
def KNN(K):
  final_result = predict(K)
  return final_result

In [ ]:
final_result = KNN(3)

In [ ]:
def accuracy(predict, actual):
  TP_TN = 0
  for i in range (len(predict)):
    if predict[i]+"." == actual[i]:
        TP_TN += 1

  return (TP_TN/len(predict))

In [ ]:
actual_classes = test_data['class']
print("accuracy : ", accuracy(final_result,actual_classes))

accuracy :  0.7917888563049853
